# Preprocesamiento de los datos - Análisis de suelos


## Importaciones

In [127]:
import numpy as np
import pandas as pd
import sqlite3

## Funciones

In [128]:
def clean_float_values(value):
    '''Limpia valores en columnas float
    
    Procesa valores string realizando las siguientes acciones:
      - Reemplaza ',' por '.'
      - Reemplaza '..' por '.'
      - Elimina el signo '>' dejando el límite superior como valor actual
      - Reemplaza los límites inferiores por la mitad de su valor
      - Reemplaza valores inválidos por NaN
    '''
    if type(value) != str:
        return value
    
    value = value.replace(',', '.')
    value = value.replace('..', '.')
    value = value.replace('>', '')
    
    if value[0] == '<':
        value = float((value[1:]).strip()) / 2        
    else:
        try:
            float(value)
        except Exception as e:
            if value in rares.keys():
                rares[value] += 1
            else:
                rares[value] = 1
            value = 'NaN'
    return value

In [129]:
def clean_string_values(value):
    value = value.replace(' ', '_')
    value = value.replace('__', '_')
    return value

In [130]:

def get_db():
    DB_CONNECTION = sqlite3.connect('data/database.db')
    DB_CURSOR = DB_CONNECTION.cursor()
    return DB_CONNECTION, DB_CURSOR

In [131]:
def replace_accented_chars(s):
    replacements = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'Ä': 'A', 'Ë': 'E', 'Ï': 'I', 'Ö': 'O', 'Ü': 'U',
        'ä': 'a', 'ë': 'e', 'ï': 'i', 'ö': 'o', 'ü': 'u',
        'ñ': 'n', 'Ñ': 'N', ',': ''}
    for k, v in replacements.items():
        s = s.replace(k, v) 
    return s

## Dataset: Análisis de suelos

### Carga de los datos

In [132]:
df_analisis = pd.read_csv('data/suelos_original.csv')

C:\Users\Islandi Navarro\AppData\Local\Temp\ipykernel_5040\1093822172.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_analisis = pd.read_csv('data/suelos_original.csv')


### Tratamiento

#### Renombrar columnas

In [133]:
oDict = {
    'numfila': 'id',
    'Departamento': 'departamento',
    'Municipio': 'municipio',
    'Cultivo': 'cultivo',
    'Estado': 'estado',
    'Tiempo Establecimiento': 'tiempo_establecimiento',
    'Topografia': 'topografia',
    'Drenaje': 'drenaje',
    'Riego': 'riego',
    'Fertilizantes aplicados': 'fertilizantes',
    'FechaAnalisis': 'fecha',
    'pH agua:suelo 2,5:1,0': 'ph',
    'Materia orgánica (MO) %': 'materia_organica',
    'Fósforo (P) Bray II mg/kg': 'fosforo',
    'Azufre (S) Fosfato monocalcico mg/kg': 'azufre',
    'Acidez (Al+H) KCL cmol(+)/kg': 'acidez',
    'Aluminio (Al) intercambiable cmol(+)/kg': 'aluminio',
    'Calcio (Ca) intercambiable cmol(+)/kg': 'calcio',
    'Magnesio (Mg) intercambiable cmol(+)/kg': 'magnesio',
    'Potasio (K) intercambiable cmol(+)/kg': 'potasio',
    'Sodio (Na) intercambiable cmol(+)/kg': 'sodio',
    'capacidad de intercambio cationico (CICE) suma de bases cmol(+)/kg': 'cice',
    'Conductividad el‚ctrica (CE) relacion 2,5:1,0 dS/m': 'ce',
    'Hierro (Fe) disponible olsen mg/kg': 'hierro_olsen',
    'Cobre (Cu) disponible mg/kg': 'cobre',
    'Manganeso (Mn) disponible Olsen mg/kg': 'manganeso',
    'Zinc (Zn) disponible Olsen mg/kg': 'zinc_olsen',
    'Boro (B) disponible mg/kg': 'boro',
    'Hierro (Fe) disponible doble \xa0cido mg/kg': 'hierro_doble_acido',
    'Cobre (Cu) disponible doble acido mg/kg': 'cobre_doble_acido',
    'Manganeso (Mn) disponible doble acido mg/kg': 'manganeso_doble_acido',
    'Zinc (Zn) disponible doble \xa0cido mg/kg': 'zinc_doble_acido', 
    'Secuencial': 'secuencial'
}
df_analisis.rename(columns=oDict, inplace=True)

#### Eliminar columnas sin valores

In [134]:
df_analisis.drop(columns=['secuencial', 'fecha'], inplace=True)

#### Formato de valores en columnas

In [135]:
# Cambiar los valores string a minúsculas
columnas = [
    'departamento', 'municipio', 'cultivo', 'estado', 'tiempo_establecimiento', 
    'topografia', 'drenaje', 'riego', 'fertilizantes'
]
for columna in columnas:
    df_analisis[columna] = df_analisis[columna].str.lower()

#### Corrección de valores float

In [136]:
rares = {}

# Limpieza de carácteres no numéricos y conversión de valores numéricos a float
columnas = [
    'ph', 'fosforo', 'azufre', 'acidez', 'aluminio', 
    'calcio', 'magnesio', 'potasio', 'sodio', 'ce', 'hierro_olsen', 
    'cobre', 'manganeso', 'zinc_olsen', 'boro', 'hierro_doble_acido',
    'cobre_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido'
]

for columna in columnas:
    try:
        df_analisis[columna] = df_analisis[columna].apply(clean_float_values)
        df_analisis[columna] = df_analisis[columna].astype(float)
    except Exception as e:
        print(str(e))
if len(rares):
    print("Valores problemáticos convertidos a NaN:\n", rares)

Valores problemáticos convertidos a NaN:
 {'MI': 2, 'ND': 235138}


#### Corrección de valores string

In [137]:
df_analisis["topografia"] = df_analisis["topografia"].str.replace(
    "error: #n/a", "no indica")

oDict = {
    'error: #n/a': "no indica",
    'buen drenaje': 'bueno',
    'regular drenaje': 'regular',
    'mal drenaje': 'malo'
}
df_analisis["drenaje"].replace(oDict, inplace=True)

In [138]:
columnas = [
    'municipio', 'departamento', 'estado', 'tiempo_establecimiento', 
    'topografia', 'drenaje', 'riego']

for columna in columnas:
    try:
        df_analisis[columna] = df_analisis[columna].apply(clean_string_values)
    except Exception as e:
        print(columna, str(e))

In [139]:
df_analisis['departamento'] = \
    df_analisis['departamento'].apply(replace_accented_chars)
df_analisis['municipio'] = \
    df_analisis['municipio'].apply(replace_accented_chars)

In [145]:
_replaces = {
    'armero_guayabal': 'armero',
    'cuaspud_carlosama': 'cuaspud',
    'cucuta': 'san_jose_de_cucuta',
    'guican': 'guican_de_la_sierra',
    'mompos': 'santa_cruz_de_mompox',
    'san_luis_de_cubarral': 'cubarral',
    'tolu_viejo': 'toluviejo'}
df_analisis['municipio'] = df_analisis['municipio'].replace(_replaces)

---

#### Eliminación de registros


Esta operación es discutible ya que esos registros pueden servir para la detección de anomalías aunque no sirvan para el modelo de predicción
```python
indices = df_analisis[
    (df_analisis["cultivo"] == "no indica") & 
    (df_analisis["estado"] == "no indica") & 
    (df_analisis["tiempo_establecimiento"] == "no indica")].index
df_analisis.drop(index=indices, axis=0, inplace=True)
```

In [141]:
# Eliminar las filas con el valor borrar en la columna municipio
df_analisis = df_analisis[df_analisis['municipio'] != 'borrar']

#### Columnas categorizadas

In [142]:
# Crear columnas dummy a partir de la columna topografia
columnas = [
    'estado', 'tiempo_establecimiento', 'topografia', 'drenaje', 'riego']

for columna in columnas:
    df_analisis = pd.get_dummies(df_analisis, columns=[columna])

# Ajustar la dummificación de las columnas compuestas
columnas = {
    'topografia_ondulado_y_pendiente' : ['topografia_ondulado', 'topografia_pendiente'],
    'topografia_plano_y_ondulado' : ['topografia_plano', 'topografia_ondulado'],
    'topografia_plano_y_pendiente' : ['topografia_plano', 'topografia_pendiente'],
    'riego_aspersión_y_goteo': ['riego_aspersión', 'riego_goteo'],
    'riego_aspersión_y_gravedad': ['riego_aspersión', 'riego_gravedad'],
    'riego_goteo_y_gravedad': ['riego_goteo', 'riego_gravedad'],
}
for key, value in columnas.items():
    indices = df_analisis[(df_analisis[key] == 1)].index
    df_analisis.loc[indices, value[0]] = 1
    df_analisis.loc[indices, value[1]] = 1
    df_analisis.drop(columns=[key], inplace=True)

#### Completar columnas faltantes

In [146]:
DB_CONNECTION, DB_CURSOR = get_db()

try:
    query = "DELETE FROM analisis"
    DB_CURSOR.execute(query)
    df_analisis.to_sql(
        'analisis', DB_CONNECTION, if_exists='append', index=False)

    query = '''
        UPDATE analisis 
        SET cod_departamento = departamentos.codigo 
        FROM departamentos
        WHERE analisis.departamento = departamentos.clean '''
    DB_CURSOR.execute(query)

    query = '''
        UPDATE analisis 
        SET cod_municipio = municipios.codigo 
        FROM municipios
        WHERE analisis.municipio = municipios.clean '''
    DB_CURSOR.execute(query)

    DB_CONNECTION.commit()
    DB_CONNECTION.close()
except Exception as e:
    print(e)
    DB_CONNECTION.close()

### Exportar conjunto de datos

In [ ]:
df_analisis.to_csv('data/estaciones_preprocesado.csv', index=False)

## Dataset: Estaciones metereológicas

### Carga de los datos

In [62]:
df_estaciones = pd.read_csv('data/estaciones_original.csv')

### Tratamiento

#### Renombrar columnas

In [63]:
oDict = {
    'Codigo': 'codigo', 
    'Nombre': 'nombre',
    'Categoria': 'categoria', 
    'Tecnologia': 'tecnologia',
    'Estado': 'estado',
    'Departamento': 'departamento',
    'Municipio': 'municipio',
    'Ubicación': 'ubicacion',
    'Altitud': 'altitud',
    'Fecha_instalacion': 'fecha_instalacion',
    'Fecha_suspension': 'fecha_suspension',
    'Area Operativa': 'area_operativa',
    'Corriente': 'corriente',
    'Area Hidrografica': 'area_hidrografica',
    'Zona Hidrografica': 'zona_hidrografica',
    'Subzona hidrografica': 'subzona_hidrografica',
    'Entidad': 'entidad'
}
df_estaciones.rename(columns=oDict, inplace=True)

#### Filtrar columnas requeridas

In [64]:
df_estaciones = df_estaciones[
    ['departamento', 'municipio', 'ubicacion', 'altitud']]

,departamento,municipio,ubicacion,altitud
0,Nariño,Ipiales,"(0.81378611, -77.66197778)",2582
1,Nariño,Contadero,"(0.93030556, -77.49119444)",2450
2,Nariño,Túquerres,"(1.07061111, -77.63688889)",3120


#### Formato de valores en columnas

In [65]:
# Cambiar los valores string a minúsculas
columnas = ['departamento', 'municipio']
for columna in columnas:
    df_estaciones[columna] = df_analisis[columna].str.lower()

df_estaciones['departamento'] = \
    df_estaciones['departamento'].apply(clean_string_values)
df_estaciones['departamento'] = \
    df_estaciones['departamento'].apply(replace_accented_chars)
df_estaciones['municipio'] = \
    df_estaciones['municipio'].apply(clean_string_values)
df_estaciones['municipio'] = \
    df_estaciones['municipio'].apply(replace_accented_chars)

#### Corrección de valores string

In [66]:
df_estaciones['municipio'] = \
    df_estaciones['municipio'].replace('cucuta', 'san_jose_de_cucuta')

#### Creación de nuevas columnas

In [67]:
df_estaciones['latitud'] = \
    df_estaciones['ubicacion'].apply(lambda x: (eval(x))[0])
df_estaciones['longitud'] = \
    df_estaciones['ubicacion'].apply(lambda x: (eval(x))[1])
df_estaciones.drop(columns=['ubicacion'], inplace=True)

#### Agrupar registros múltiples para el mismo municipio

In [68]:
df_estaciones = \
    df_estaciones.groupby(['departamento', 'municipio'])[
        ['altitud', 'latitud', 'longitud']].mean().reset_index()

#### Completar columnas faltantes

In [70]:
try:
    DB_CONNECTION, DB_CURSOR = get_db()

    query = "DELETE FROM estaciones;"
    DB_CURSOR.execute(query)

    df_estaciones.to_sql(
        'estaciones', con=DB_CONNECTION, if_exists='append', index=False)

    query = '''
        UPDATE estaciones 
        SET cod_departamento = departamentos.codigo 
        FROM departamentos
        WHERE estaciones.departamento = departamentos.clean '''
    DB_CURSOR.execute(query)

    query = '''
        UPDATE estaciones 
        SET cod_municipio = municipios.codigo 
        FROM municipios
        WHERE estaciones.municipio = municipios.clean '''
    DB_CURSOR.execute(query)

    DB_CONNECTION.commit()
    DB_CONNECTION.close()
except Exception as e:
    print(e)
    DB_CONNECTION.close()

In [76]:
DB_CONNECTION, DB_CURSOR = get_db()

df_estaciones = pd.read_sql_query(
    'SELECT * FROM estaciones', con=DB_CONNECTION, index_col='id')

DB_CONNECTION.close()

### Exportar conjunto de datos

In [ ]:
df_estaciones.to_csv('data/estaciones_preprocesado.csv', index=False)

---
Fin del notebook